<a href="https://colab.research.google.com/github/weix20/CUH604CMD-Assignment/blob/main/Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is a binary classification problem (label ∈ {0,1}).
 - We use a Feedforward Neural Network (FNN: Dense + ReLU hidden, Sigmoid output)
 Trained with Adam + binary_crossentropy;
 - features are standardized.



In [1]:
# Import required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.callbacks import EarlyStopping

# Automatically load the "/content/data.csv" file
df = pd.read_csv('/content/data.csv')

# Show the first few rows of the dataset
print(df.head())

# Prepare dataset
# Separate features and labels
X = df.drop(columns=['y'])  # Features
y = df['y']  # Labels

# Split the dataset into training and testing sets (80% for training, 20% testing)
# The random_state ensures that the split is reproducible
# stratify=y keeps class ratio
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=24, stratify=y
)

# Standardize the data (important for neural networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build a Feedforward Neural Network (FNN)
def create_model(layers_count=3, neurons_per_layer=64, dropout_rate=0.2, learning_rate=0.001):
    model = models.Sequential()

    # Input layer matches the number of features
    model.add(layers.InputLayer(shape=(X_train.shape[1],)))


    # Hidden layers: Dense + ReLU + Dropout
    for _ in range(layers_count):
        model.add(layers.Dense(neurons_per_layer, activation='relu'))
        model.add(layers.Dropout(dropout_rate))

    # Output layer: 1 neuron with Sigmoid for binary classification
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile with Adam optimizer and binary crossentropy loss
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',  # Binary classification loss
                  metrics=['accuracy'])

    return model

# Define three models with different hyperparameters
# Model 1: Default architecture
model1 = create_model(layers_count=3, neurons_per_layer=64, dropout_rate=0.2, learning_rate=0.001)

# Model 2: Increased neurons and layers
model2 = create_model(layers_count=4, neurons_per_layer=128, dropout_rate=0.3, learning_rate=0.0005)

# Model 3: Different architecture with fewer layers and neurons
model3 = create_model(layers_count=2, neurons_per_layer=32, dropout_rate=0.1, learning_rate=0.002)

# To add early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train all models
print("\n===== TRAINING Model 1 =====")
history1 = model1.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test),
                      callbacks=[early_stopping], verbose=1)

print("\n===== TRAINING Model 2 =====")
history2 = model2.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test),
                      callbacks=[early_stopping], verbose=1)

print("\n===== TRAINING Model 3 =====")
history3 = model3.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test),
                      callbacks=[early_stopping], verbose=1)

# Evaluate all models on the test set
print("\n----- EVALUATING on TEST -----")
loss1, accuracy1 = model1.evaluate(X_test, y_test)
loss2, accuracy2 = model2.evaluate(X_test, y_test)
loss3, accuracy3 = model3.evaluate(X_test, y_test)

# Predict the results on the test set
proba1 = model1.predict(X_test, verbose=0).ravel()
proba2 = model2.predict(X_test, verbose=0).ravel()
proba3 = model3.predict(X_test, verbose=0).ravel()

y_pred1 = (model1.predict(X_test) > 0.5).astype(int)  # Convert binary output to 0 or 1
y_pred2 = (model2.predict(X_test) > 0.5).astype(int)
y_pred3 = (model3.predict(X_test) > 0.5).astype(int)

# Calculate precision, recall, F1-score, and ROC AUC
precision1 = precision_score(y_test, y_pred1)
recall1 = recall_score(y_test, y_pred1)
f1_1 = f1_score(y_test, y_pred1)
roc_auc1 = roc_auc_score(y_test, y_pred1)

precision2 = precision_score(y_test, y_pred2)
recall2 = recall_score(y_test, y_pred2)
f1_2 = f1_score(y_test, y_pred2)
roc_auc2 = roc_auc_score(y_test, y_pred2)

precision3 = precision_score(y_test, y_pred3)
recall3 = recall_score(y_test, y_pred3)
f1_3 = f1_score(y_test, y_pred3)
roc_auc3 = roc_auc_score(y_test, y_pred3)

# Print results for each model
print(f"Model 1 - Loss: {loss1:.4f}, Accuracy: {accuracy1:.4f}, Precision: {precision1:.4f}, Recall: {recall1:.4f}, F1-score: {f1_1:.4f}, ROC AUC: {roc_auc1:.4f}")
print(f"Model 2 - Loss: {loss2:.4f}, Accuracy: {accuracy2:.4f}, Precision: {precision2:.4f}, Recall: {recall2:.4f}, F1-score: {f1_2:.4f}, ROC AUC: {roc_auc2:.4f}")
print(f"Model 3 - Loss: {loss3:.4f}, Accuracy: {accuracy3:.4f}, Precision: {precision3:.4f}, Recall: {recall3:.4f}, F1-score: {f1_3:.4f}, ROC AUC: {roc_auc3:.4f}")

# Choose the best model based on accuracy
best_model = None
best_accuracy = max(accuracy1, accuracy2, accuracy3)

if best_accuracy == accuracy1:
    best_model = model1
elif best_accuracy == accuracy2:
    best_model = model2
else:
    best_model = model3

# Final performance evaluation
print(f"The best model is Model {['1', '2', '3'][[accuracy1, accuracy2, accuracy3].index(best_accuracy)]} with accuracy: {best_accuracy:.4f}")

   x0  x1  x2  x3  x4  x5  x6  x7  x8  x9  x10  x11  x12  x13  x14  y
0   0   1   0   0   0   0   0   0   0   0    0    0    0    0    0  1
1   0   0   1   0   0   0   0   0   0   0    0    0    0    0    0  0
2   0   0   0   1   0   0   0   0   0   0    0    0    0    0    0  1
3   0   0   0   0   1   0   0   0   0   0    0    0    0    0    0  0
4   0   0   0   0   0   1   0   0   0   0    0    0    0    0    0  1

===== TRAINING Model 1 =====
Epoch 1/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.5019 - loss: 0.7005 - val_accuracy: 0.5009 - val_loss: 0.6939
Epoch 2/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.5033 - loss: 0.6943 - val_accuracy: 0.4966 - val_loss: 0.6934
Epoch 3/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5133 - loss: 0.6932 - val_accuracy: 0.5026 - val_loss: 0.6936
Epoch 4/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5084 - loss: 0.6932 - val_accuracy: 0.4971 - val_loss: 0.6939
Epoch 5/50
819/819 ━━━━━━━━━━━━━━━━━━